<a href="https://colab.research.google.com/github/Amplil/nlp-test/blob/master/opmnt_library_transformer_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 初期化

In [2]:
cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [3]:
ls

data@                                      pred.txt
new_colab2.ipynb                           toy-ende@
opmnt_library_test.ipynb                   Transformer@
opmnt_library_transformer_translate.ipynb  transformer_kyoto_ja_en.ipynb
packages@                                  transformer_test.ipynb


In [4]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/packages")
import slacknotice # オリジナルモジュール

In [5]:
pip install OpenNMT-py

     |████████████████████████████████| 216 kB 5.1 MB/s 
     |████████████████████████████████| 73 kB 1.6 MB/s 
     |████████████████████████████████| 16.0 MB 117 kB/s 
     |████████████████████████████████| 56 kB 4.2 MB/s 
     |████████████████████████████████| 1.2 MB 28.0 MB/s 
  Attempting uninstall: torchtext
    Found existing installation: torchtext 0.10.0
    Uninstalling torchtext-0.10.0:
      Successfully uninstalled torchtext-0.10.0


In [6]:
import torch
import torch.nn as nn
from argparse import Namespace
from collections import defaultdict, Counter

In [7]:
import onmt
from onmt.inputters.inputter import _load_vocab, _build_fields_vocab, get_fields, IterOnDevice
from onmt.inputters.corpus import ParallelCorpus
from onmt.inputters.dynamic_iterator import DynamicDatasetIter
from onmt.translate import GNMTGlobalScorer, Translator, TranslationBuilder
from onmt.utils.misc import set_random_seed

In [8]:
# enable logging
from onmt.utils.logging import init_logger, logger
init_logger()

<RootLogger root (INFO)>

# モデルの読み込み

In [9]:
import onmt.opts as opts
from onmt.utils.parse import ArgumentParser
#from onmt.opts import dynamic_prepare_opts

parser = ArgumentParser(description='translate.py')
#dynamic_prepare_opts(parser, build_vocab_only=True)
opts.translate_opts(parser)

base_args = (["-model", "data/kftt-data-1.0/run/model_step_15000.pt", "-src", "data/en_sample.txt", "-output", "data/kftt-data-1.0/out.txt", "-gpu", "0", "-verbose","-attn_debug"])
opt, unknown = parser.parse_known_args(base_args)
opt

Namespace(align_debug=False, alpha=0.0, attn_debug=True, avg_raw_probs=False, ban_unk_token=False, batch_size=30, batch_type='sents', beam_size=5, beta=-0.0, block_ngram_repeat=0, coverage_penalty='none', data_type='text', dump_beam='', fp32=False, gpu=0, ignore_when_blocking=[], int8=False, length_penalty='none', log_file='', log_file_level='0', max_length=100, max_sent_length=None, min_length=0, models=['data/kftt-data-1.0/run/model_step_15000.pt'], n_best=1, output='data/kftt-data-1.0/out.txt', phrase_table='', random_sampling_temp=1.0, random_sampling_topk=0, random_sampling_topp=0.0, ratio=-0.0, replace_unk=False, report_align=False, report_time=False, seed=-1, shard_size=10000, src='data/en_sample.txt', src_feats=None, stepwise_penalty=False, tgt=None, tgt_prefix=False, verbose=True)

In [10]:
from onmt.translate.translator import build_translator

ArgumentParser.validate_translate_opts(opt)
logger = init_logger(opt.log_file)

translator = build_translator(opt, logger=logger, report_score=True)

In [11]:
"""
# out_fileの作成
if out_file is None:
    out_file = codecs.open(opt.output, "w+", "utf-8")
"""
load_test_model = (
    onmt.decoders.ensemble.load_test_model
    if len(opt.models) > 1
    else onmt.model_builder.load_test_model
)
vocab_fields, model, model_opt = load_test_model(opt) # fields->vocab_fields

scorer = onmt.translate.GNMTGlobalScorer.from_opt(opt)

In [12]:
#def load_test_model(opt, model_path=None)
"""
from onmt.utils.parse import ArgumentParser
import onmt.inputters as inputters
from onmt.utils.misc import use_gpu

model_path="data/kftt-data-1.0/run/model_step_15000.pt"

checkpoint = torch.load(model_path,map_location=lambda storage, loc: storage)

model_opt = ArgumentParser.ckpt_model_opts(checkpoint['opt'])
ArgumentParser.update_model_opts(model_opt)
ArgumentParser.validate_model_opts(model_opt)
fields = checkpoint['vocab']

# Avoid functionality on inference
model_opt.update_vocab = False

model = onmt.model_builder.build_base_model(model_opt, fields, use_gpu(opt), checkpoint,
                            opt.gpu)
if opt.fp32:
    model.float()
elif opt.int8:
    if opt.gpu >= 0:
        raise ValueError(
            "Dynamic 8-bit quantization is not supported on GPU")
    torch.quantization.quantize_dynamic(model, inplace=True)
model.eval()
model.generator.eval()
"""

'\nfrom onmt.utils.parse import ArgumentParser\nimport onmt.inputters as inputters\nfrom onmt.utils.misc import use_gpu\n\nmodel_path="data/kftt-data-1.0/run/model_step_15000.pt"\n\ncheckpoint = torch.load(model_path,map_location=lambda storage, loc: storage)\n\nmodel_opt = ArgumentParser.ckpt_model_opts(checkpoint[\'opt\'])\nArgumentParser.update_model_opts(model_opt)\nArgumentParser.validate_model_opts(model_opt)\nfields = checkpoint[\'vocab\']\n\n# Avoid functionality on inference\nmodel_opt.update_vocab = False\n\nmodel = onmt.model_builder.build_base_model(model_opt, fields, use_gpu(opt), checkpoint,\n                            opt.gpu)\nif opt.fp32:\n    model.float()\nelif opt.int8:\n    if opt.gpu >= 0:\n        raise ValueError(\n            "Dynamic 8-bit quantization is not supported on GPU")\n    torch.quantization.quantize_dynamic(model, inplace=True)\nmodel.eval()\nmodel.generator.eval()\n'

In [13]:
model

NMTModel(
  (encoder): TransformerEncoder(
    (embeddings): Embeddings(
      (make_embedding): Sequential(
        (emb_luts): Elementwise(
          (0): Embedding(13639, 512, padding_idx=1)
        )
        (pe): PositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (transformer): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiHeadedAttention(
          (linear_keys): Linear(in_features=512, out_features=512, bias=True)
          (linear_values): Linear(in_features=512, out_features=512, bias=True)
          (linear_query): Linear(in_features=512, out_features=512, bias=True)
          (softmax): Softmax(dim=-1)
          (dropout): Dropout(p=0.1, inplace=False)
          (final_linear): Linear(in_features=512, out_features=512, bias=True)
        )
        (feed_forward): PositionwiseFeedForward(
          (w_1): Linear(in_features=512, out_features=2048, bias=True)
          (w_2): Linear(in_features=2048, o

In [14]:
import onmt.inputters as inputters
src_reader = inputters.str2reader[opt.data_type].from_opt(opt)
tgt_reader = inputters.str2reader["text"].from_opt(opt)

In [15]:
from onmt.translate.translator import max_tok_len

src_data = {
    "reader": src_reader, # self.src_reader->src_reader
    "data": opt.src, # src->opt.src
    "features": {} # src_feats->{}
}
tgt_data = {
    "reader": tgt_reader, # self.tgt_reader->tgt_reader
    "data": opt.tgt, # tgt->opt.tgt
    "features": {}
}
_readers, _data = inputters.Dataset.config(
    [("src", src_data), ("tgt", tgt_data)]
)

dataset = inputters.Dataset( # data->dataset
    vocab_fields, # self.fields->vocab_fields
    readers=_readers,
    data=_data,
    sort_key=inputters.str2sortkey[opt.data_type], # self->opt
    filter_pred=None, # self._filter_pred -> None
)

data_iter = inputters.OrderedIterator(
    dataset=dataset, # data->dataset
    device="cuda", # self._dev->"cuda"
    batch_size=opt.batch_size, # batch_size->opt.batch_size
    batch_size_fn=max_tok_len if opt.batch_type == "tokens" else None, # batch_type->opt.batch_type
    train=False,
    sort=False,
    sort_within_batch=True,
    shuffle=False,
)

builder = onmt.translate.TranslationBuilder(data=dataset, fields=vocab_fields)

In [16]:
from onmt.constants import DefaultTokens
from onmt.utils.misc import tile, set_random_seed, report_matrix

# アテンションデバッグをON->今後はopt.attn_debugを使う
# attn_debug=True
for batch in data_iter:
    batch_data = translator.translate_batch(
        batch, dataset.src_vocabs, opt.attn_debug
    )
    translations = builder.from_batch(batch_data)

    for trans in translations:
        print(trans.log(0))
        
        if opt.attn_debug:
            preds = trans.pred_sents[0]
            preds.append(DefaultTokens.EOS)
            attns = trans.attns[0].tolist()
            if translator.data_type == "text":
                srcs = trans.src_raw
            else:
                srcs = [str(item) for item in range(len(attns[0]))]
            output = report_matrix(srcs, preds, attns)
            if translator.logger:
                translator.logger.info(output)
            else:
                #os.write(1, output.encode("utf-8"))
                print(output)
    break

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)
[2021-11-07 08:58:55,707 INFO]                   Tom      wants          a        cat          . 
     <unk>  0.0810000  0.1024835  0.2358939 *0.3071590  0.2734635 
         は  0.0732654  0.2247687  0.1639529  0.1544976 *0.3835154 
     <unk>  0.0123236  0.0434103  0.2558491 *0.5182255  0.1701915 
         を  0.0581807  0.1314416  0.1406691  0.1174425 *0.5522662 
        求め  0.0094350  0.0870038  0.1193642  0.1103887 *0.6738083 
         た  0.0225705 


SENT 0: ['Tom', 'wants', 'a', 'cat', '.']
PRED 0: <unk> は <unk> を 求め た 。
PRED SCORE: -5.6415



# アテンションの利用

In [17]:
opt.attn_debug

True